# Elise Hu: 2019 books

Quick analysis of the 52 books she read last year. Published [here](http://heyelise.com/2020/01/01/52-books-i-read-in-2019-charted/). 

### Load Python tools 

In [1]:
import pandas as pd
import geopandas as gpd
from urllib.request import urlopen 
import pyarrow
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
import altair_latimes as lat

In [2]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [3]:
plt.rcParams['figure.figsize'] = (16,8)

In [4]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
pd.options.display.float_format = '{:,.2f}'.format

### Read book data from Elise

In [6]:
src = pd.read_excel('input/elise-2019-books.xlsx')

In [7]:
src.dtypes

Order                            int64
Title                           object
Author                          object
Pages                            int64
URL                             object
gender                          object
type                            object
Subgenre                        object
Avg GoodReads Rating           float64
Started                 datetime64[ns]
date                    datetime64[ns]
Faves                           object
dtype: object

### Clean up field headers, data types

In [8]:
src.columns = src.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [9]:
src['year'] = src['date'].dt.year.astype(str).str.replace('.0','',regex=False)

In [10]:
src['type'] = src['type'].str.replace(' ', '-')

In [11]:
src['days_to_read'] = src['date'] - src['started']

### Make gender more descriptive, calculate number of reading days

In [12]:
src['gender_description'] = src.gender.replace({'M': 'Male', 'F': 'Female'})

In [13]:
src['days_to_read'] = src['days_to_read'].dt.days

### Now start fresh with a dataframe called 'books'

In [14]:
books = pd.DataFrame(src)

### When did Elise finish her books?

In [15]:
# DESKTOP 
alt.Chart(books).mark_tick(thickness=2,size=20).encode(
    x=alt.X('date',axis=alt.Axis(tickCount=12, format='%b', tickColor='#ffffff'), title=''),
    y=alt.Y('year',axis=alt.Axis(tickCount=0, tickColor='#ffffff'), title=' ')
).configure_axis(
    grid=False).properties(height=40, width=700)

alt.Chart(...)

In [16]:
# MOBILE 
alt.Chart(books).mark_tick(thickness=2,size=20).encode(
    x=alt.X('date',axis=alt.Axis(tickCount=12, format='%b', tickColor='#ffffff'), title=''),
    y=alt.Y('year',axis=alt.Axis(tickCount=0, tickColor='#ffffff'), title=' ')
).configure_axis(
    grid=False).properties(height=40, width=320)

alt.Chart(...)

### Tinkering with more designs for books read by subgenre and month

In [17]:
#DESKTOP
alt.Chart(books).mark_tick(thickness=3,size=10).encode(
    x=alt.X('date',axis=alt.Axis(tickCount=12, format='%b', tickColor='#ffffff'), title=''),
    y=alt.Y('subgenre',axis=alt.Axis(tickCount=0, tickColor='#ffffff'), title=' ')
).configure_axis(grid=False).properties(height=400, width=700)

alt.Chart(...)

In [18]:
# mobile
alt.Chart(books).mark_tick(thickness=2,size=10).encode(
    x=alt.X('date',axis=alt.Axis(tickCount=12, format='%b', tickColor='#ffffff'), title=''),
    y=alt.Y('subgenre',axis=alt.Axis(tickCount=0, tickColor='#ffffff'), title=' ')
).configure_axis(grid=False).properties(height=400, width=320)

alt.Chart(...)

---

### Let's try books by author gender and month

In [19]:
#DESKTOP
alt.Chart(books).mark_tick(thickness=4,size=20).encode(
    x=alt.X('date',axis=alt.Axis(tickCount=12, format='%b', tickColor='#ffffff'), title=''),
    y=alt.Y('gender_description',axis=alt.Axis(tickCount=0, tickColor='#ffffff'), title=' '),
    color=alt.Color('gender_description', legend=None)
).configure_axis(
    grid=False).properties(height=100, width=700)

alt.Chart(...)

In [20]:
# MOBILE
alt.Chart(books).mark_tick(thickness=2,size=20).encode(
    x=alt.X('date',axis=alt.Axis(tickCount=12, format='%b', tickColor='#ffffff'), title=''),
    y=alt.Y('gender_description',axis=alt.Axis(tickCount=0, tickColor='#ffffff'), title=' '),
    color=alt.Color('gender_description', legend=None)
).configure_axis(
    grid=False).properties(height=100, width=320)

alt.Chart(...)

### Books by type and month

In [21]:
# DESKTOP 
alt.Chart(books).mark_tick(thickness=3,size=20).encode(
    x=alt.X('date',axis=alt.Axis(tickCount=12, format='%b', tickColor='#ffffff'), title=''),
    y=alt.Y('type',axis=alt.Axis(tickCount=0, tickColor='#ffffff'), title=' '),
    color=alt.Color('type', legend=None)
).configure_axis(
    grid=False).properties(height=200, width=700, title='Books, by type and date finished')

alt.Chart(...)

In [22]:
# MOBILE
alt.Chart(books).mark_tick(thickness=2,size=20).encode(
    x=alt.X('date',axis=alt.Axis(tickCount=12, format='%b', tickColor='#ffffff'), title=''),
    y=alt.Y('type',axis=alt.Axis(tickCount=0, tickColor='#ffffff'), title=' '),
    color=alt.Color('type', legend=None)
).configure_axis(
    grid=False).properties(height=200, width=320, title='Books, by type and date finished')

alt.Chart(...)

---

### Is there a strong relationship between the number of pages and days to read? No

In [23]:
# Remove handful of books that took months to finish
books_month = books[books['days_to_read'] < 30]

In [24]:
books_corr = books_month[['pages', 'days_to_read']]

In [25]:
corr = books_corr.corr(method ='pearson')

In [26]:
print(corr)

              pages  days_to_read
pages          1.00          0.34
days_to_read   0.34          1.00


In [27]:
#DESKTOP 
alt.Chart(books_month).mark_circle(size=60).encode(
    x=alt.X('days_to_read:Q', title='Days to read', axis=alt.Axis(tickCount=6)),
    y=alt.Y('pages:Q', title='Number of pages', axis=alt.Axis(tickCount=5)),
    tooltip=['title:N', 'author:O', 'avg_goodreads_rating:Q', 'days_to_read:Q']
).properties(width=500, height=500)

alt.Chart(...)

In [28]:
# MOBILE
alt.Chart(books_month).mark_circle(size=60).encode(
    x=alt.X('days_to_read:Q', title='Days to read', axis=alt.Axis(tickCount=6)),
    y=alt.Y('pages:Q', title='Number of pages', axis=alt.Axis(tickCount=5)),
    tooltip=['title:N', 'author:O', 'avg_goodreads_rating:Q', 'days_to_read:Q']
).properties(width=320, height=320)

alt.Chart(...)

---

### Which types of books did Elise read most?

In [29]:
book_type = books.groupby(['type', 'year']).agg('size').reset_index(name='count').sort_values(by='count', ascending=False)

In [30]:
book_type.head()

,type,year,count
1,Non-Fiction,2019,28
0,Fiction,2019,23
2,Poetry,2019,1


In [31]:
book_type['share'] = (book_type['count'] / 52)*100

In [32]:
book_type.head()

,type,year,count,share
1,Non-Fiction,2019,28,53.85
0,Fiction,2019,23,44.23
2,Poetry,2019,1,1.92


In [33]:
book_type_chart = alt.Chart(book_type).mark_bar().encode(
    x=alt.X('sum(count)', stack="normalize",  title='Share of books read by genre', \
            axis=alt.Axis(format='%',tickCount=6)),
    y=alt.Y('year',title=''),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'type',
      sort='ascending'
    ),
    color=alt.Color('type', legend=alt.Legend(orient="top", title='Book type'))
)

(book_type_chart).properties(height=40,width=700)

alt.Chart(...)

In [34]:
(book_type_chart).properties(height=40,width=320)

alt.Chart(...)

---

### Which genre of books did Elise read most?

In [35]:
book_genre = books.groupby('subgenre').agg('size').reset_index(name='count')\
.sort_values(by='count', ascending=False)
book_genre.head()

,subgenre,count
2,Contemporary,10
10,Memoir,5
3,Essays,5
4,Feminism,5
7,History,4


### Books read by sub-genre

In [36]:
#DESKTOP
book_genre_chart = alt.Chart(book_genre).mark_bar().encode(
    y=alt.Y("subgenre:N", title=' ',
           sort=alt.EncodingSortField(
            field="count:Q",  # The field to use for the sort
            op="sum",  # The operation to run on the field prior to sorting
            order="ascending"  # The order to sort in
        )),
    x=alt.X("count:Q", title=' ', axis=alt.Axis(tickCount=6, format=''))
)

book_genre_chart.properties(height=500, width=700, title='')

alt.Chart(...)

In [37]:
#MOBILE 
book_genre_chart.properties(height=500, width=320, title='')

alt.Chart(...)

---

### Memior as share of all non-fiction books read

In [38]:
nonfiction = books[books['type'] == 'Non-Fiction'].groupby('subgenre').agg('size').reset_index(name='count')\
.sort_values(by='count', ascending=False)
nonfiction.head(20)

,subgenre,count
1,Essays,5
5,Memoir,5
2,Feminism,4
4,History,4
9,Science,2
11,Writing,2
0,Business,1
3,Graphic Novel,1
6,Parenting,1
7,Pop Culture,1


In [39]:
nonfiction['share'] = (nonfiction['count'] / 28) * 100

In [40]:
nonfiction.head()

,subgenre,count,share
1,Essays,5,17.86
5,Memoir,5,17.86
2,Feminism,4,14.29
4,History,4,14.29
9,Science,2,7.14


### What share of the books Elise read were by female authors? 

In [41]:
book_gender = books.groupby(['gender_description', 'year'])\
.agg('size').reset_index(name='count').sort_values(by='count', ascending=False)

In [42]:
book_gender.head()

,gender_description,year,count
0,Female,2019,33
1,Male,2019,19


In [43]:
book_gender['share'] = (book_gender['count'] / 52)*100

### How does that gender breakdown look in a bar chart?

In [44]:
book_gender_chart = alt.Chart(book_gender).mark_bar().encode(
    x=alt.X('sum(count)', stack="normalize",  title='Share of books read by author gender',\
            axis=alt.Axis(format='%',tickCount=6)),
    y=alt.Y('year',title='',
            sort=alt.EncodingSortField(
            field="count:N",  # The field to use for the sort
            op="count",  # The operation to run on the field prior to sorting
            order="descending"  # The order to sort in)
            )),
    color=alt.Color('gender_description', legend=None)
)

book_gender_chart_text = book_gender_chart.mark_text(
    align='right',
    baseline='middle',
    dx=-10
).encode(text=alt.Text('gender_description'), 
    color=alt.condition(
        alt.datum.year == '2019',
        alt.value('white'),
        alt.value('white')
    )
)

(book_gender_chart + book_gender_chart_text).properties(height=30,width=700)

alt.LayerChart(...)

In [45]:
(book_gender_chart + book_gender_chart_text).properties(height=30,width=320)

alt.LayerChart(...)

---

### Page counts by subgenre

In [46]:
books.groupby(['subgenre']).agg({ 'pages': 'sum' }).reset_index()\
.sort_values(by='pages',ascending=False).head(10)

,subgenre,pages
2,Contemporary,2672
10,Memoir,1818
4,Feminism,1356
7,History,1186
3,Essays,1186
20,Young Adult,1088
14,Science,912
16,Short Stories,707
19,Writing,519
1,Classics,517


### Ratings by subgenre

In [47]:
books.groupby(['subgenre']).agg({ 'avg_goodreads_rating': 'mean' }).reset_index()\
.sort_values(by='avg_goodreads_rating',ascending=False).head(10)

,subgenre,avg_goodreads_rating
18,True Crime,4.48
5,Graphic Novel,4.30
14,Science,4.29
11,Parenting,4.26
19,Writing,4.23
1,Classics,4.19
13,Religion,4.05
0,Business,4.04
10,Memoir,4.03
7,History,4.02


### Favorite books by Goodreads rating

In [48]:
books_slim = books[['author', 'title', 'type', 'avg_goodreads_rating']][books['faves'] == 'Y']\
.sort_values(by='avg_goodreads_rating', ascending=False).head()

In [49]:
books_slim

,author,title,type,avg_goodreads_rating
41,Lori Gottlieb,Maybe You Should Talk to Somebody,Non-Fiction,4.40
25,Jia Tolentino,Trick Mirror,Non-Fiction,4.33
40,Adele Faber and Elaine Mazlish,How to Talk So Your Kids Will Listen,Non-Fiction,4.26
28,Taffy Brodesser-Akner,Fleishman Is In Trouble,Fiction,4.10
21,Leslie Jamison,The Recovering: Intoxication and its Aftermath,Non-Fiction,3.96


In [50]:
book_ratings = books.groupby(['type']).agg({'avg_goodreads_rating': 'mean'}).reset_index()

In [51]:
book_ratings

,type,avg_goodreads_rating
0,Fiction,3.79
1,Non-Fiction,4.07
2,Poetry,4.22


---

### Export books list to CSV

In [52]:
books.to_csv('output/books.csv')